In [17]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt

from utils import *

# <i class="fas fa-circle" style="color: #f0ad4e;"></i> Introduction to PCA

In this exercise you will create a basic PCA implementation that you will use for the next exercise.

## Implementing PCA

Your first task is to implement the PCA method as well as functions for
transforming to and from the space defined by the principal components.
But first, a quick recap of the terminology to minimize confusion.

Principal component analysis is about finding a linear transformation
that reduces the number of dimensions used to represent samples while
destroying as little of the variation as possible. PCA is defined by
$\Phi_{:k}$, an $M\times k$ matrix representing a linear transformation from
vectors in $M$-dimensional real space to $k$-dimensional latent space. We have the
following transformations

$$ 
b = \Phi_{:k}^\top x, 
$$

$$
x + \epsilon = \Phi_{:k} b,
$$

where
$x\in\mathbb{R}^M$ is the input vector and $y\in\mathbb{R}^K$ is the
embedded vector. As shown in the second equation, it is possible to reconstruct $x$ with some amount of error $\epsilon$. To find $\Phi$, we use the _eigenvectors_ of the covariance matrix of our data matrix $W$ where each row $i$ is a sample $x_i \in \mathbb{R}^M$. The eigenvectors are sorted by their associated eigenvalues which represent the variance of each dimension in latent space. Selecting the $k$ first columns (we use the notation $\Phi_{:k}$) results in a transformation that reduces the dimensionality of the latent space to $k$ dimensions. 

1.  <i class="fas fa-code"></i> **Setup:** Create a script file or notebook for this exercise. In it, start by loading the face shapes and images using `dset.face_shapes_data`.

2.  <i class="fas fa-code"></i> **Implement PCA:** Create a function that calculates and returns the
    principle components of the shapes dataset. Use the method described
    above where the eigenvectors of the covariance matrix is used.
    **Make sure to center the samples (subtract the mean before
    calculating the covariance matrix)**.

3.  <i class="fas fa-code"></i> **Implement transformations:** Create two functions, one for
    transforming from feature space to principal component space
    (eqaution {eq}`trans`) and one for transforming from principal
    component space to feature space
    (equation {eq}`inv`). You have to subtract the $\mu$ vector when
    transforming to the principal component space and add it again when
    transforming back to feature space. You may use the following
    modified equations for reference:

$$
b = \Phi_{:k}^\top(x-\mu)
$$  

$$
x = \Phi_{:k} b + \mu
$$

```{tip}
The reading material for the PCA lecture contains an excellent [tutorial](https://sebastianraschka.com/Articles/2014_pca_step_by_step.html) on how this can be done, but remember that copying is not allowed!!
```

```{tip}
Some of the later tasks will be easier if you return all 146 principle components. You can then create another function for extracting $n$ components to generate $\Phi$.
```

In [18]:
# Load data
path = './db'
shapes, images = face_shape_data(path)
print(shapes.shape)
shapes = shapes.T
print(shapes.shape) # in pca each row represent a person so we got 146 entites

120
(120, 146)
(146, 120)


In [19]:
# Write your implementation here.
#print(shapes[0])
# PCA
def mean(X):
    mean_vector = []

    for i in range(X.shape[0]):
        tmp_mean_vector = np.mean(shapes[i, :])
        mean_vector.append(tmp_mean_vector)

    mean_vector = np.array(mean_vector)
   
    return mean_vector
# print(mean(shapes))
    
def eigns(X,k):
    n,m = np.shape(X)
    
    # Calculating the mean of the shapes dataset
    #mean_x = np.mean(X)
    
    # Representing the mean to correspond the shapes dataset
    #y = np.ones((n, m), dtype=int)
    #mean_of_x = np.dot(mean_x,y)
    #print(mean_x)
    
    mean_vector = mean(X)
    mean_ = np.array([mean_vector] * 120)
    # Subtract the mean from the shapes dataset
    print(mean_)
    B = shapes - mean_.T #centering the data
    
    # Covariant matrix
    #Cov_matrix = np.dot(B.T,B)
    Cov_matrix = np.cov(B)
   
    # Calculate eigenvalues and eigenvectors of covariant matrix)
    eigenvalues, eigenvectors = np.linalg.eig(Cov_matrix)
    
     #svd
    #U, S, V = np.linalg.svd(Cov_matrix)
    
    return eigenvalues, eigenvectors, B, mean

#print(eigns(shapes,2)) 

def pca(X,k):
    
    eigenvalues, eigenvectors, B, mean_of_x = eigns(X,k)
    
     # Make a list of (eigenvalue, eigenvector) tuples
    eig_pairs = [(np.abs(eigenvalues[i]), eigenvectors[:,i]) for i in range(len(eigenvalues))]

    # Sort the (eigenvalue, eigenvector) tuples from high to low
    eig_pairs.sort(key=lambda x: x[0], reverse=True) 
    
    #Reduce dimensionality
    pc = np.hstack([eig_pairs[i][1].reshape(146,1) for i in range(k)])
   # print('Matrix W:\n', pc.shape)
    
    # Principal component mxk dimension 
    #pc =  eigenvectors[:,0:k]  
    
    return eigenvalues, eigenvectors, pc, B, mean_of_x, eig_pairs
    

#print(pca(shapes,2))


def feature_space_to_pc(x,k):
    va,ve,pc, B, mean_of_x, eig_pairs = pca(x,k) # pc: nxk , B: nxm, mean_of_x: nxm
    # B is (x - mean)                
    
    b =  np.dot(pc.T, B) # kxn * nxm -> kxm
                        
    return b

#print(feature_space_to_pc(shapes,2))    

def pc_to_feature_space(x,k):
    eigenvalues, eigenvectors, pc, B, mean_of_x, eig_pairs = pca(x,k) # pc: nxk , B: nxm, mean_of_x: nxm
    b = feature_space_to_pc(x,k) # kxm
                                
    
    #feature_space
    X =  np.dot(pc, b) # nxk * kxm -> nxm
    feature_space = X + mean_of_x.T
    
    return feature_space

#pc_to_feature_space(shapes,2)   

## Evaluating precision

As described above, using PCA to transform a sample $x$ to a principal
component space and back again likely results in an error $\epsilon$, called the _reconstruction error_. In this task you will implement a
method for calculating this error and use it to test the effect of
increasing or decreasing the number of principal components used.

When solving regression problems, the error is typically measured as the
average distance error, otherwise known as root mean square error
(RMSE). This is also used when calculating the construction error. For
reference, the RMSE is

$$RMSE(x, \widetilde{x}) = \sqrt{\frac{1}{N}\sum_i (x_i-\widetilde{x}_i)^2},$$

where $x$, $\widetilde{x}$ are the original and transformed samples
respectively and $N$ is the total number of samples $x_i$.

Another method for evaluating PCA models is to look at the eigenvalues,
where eigenvalue $i$ is denoted $\lambda^{(i)}$. The eigenvalues explain
the variance of each dimension when that data has been transformed by
PCA. The sum of all eigenvalues $\lambda^{(1)}+\dots+\lambda^{(n)}$ is
equal to the total variance of the data. By comparing all the
eigenvalues we can calculate:

(1) **Proportional variance:** What proportion of the total variance is
explained by a single component. The following formula can be used

$$\frac{\lambda^{(i)}}{\lambda^{(1)} + \dots + \lambda^{(n)}}$$

(2) **Cumulative proportional variance:** What cumulative proportion of
the total variance is explained by the first $k$ components.

$$\frac{\lambda^{(1)} + \dots + \lambda^{(k)}}{\lambda^{(1)} + \dots + \lambda^{(n)}}$$


1.  **<i class="fas fa-code"></i> Calculate reconstruction error:** Implement a function in your
    script that calculates the reconstruction error given a dataset $X$,
    principle components $\Phi$, and a mean vector $\mu$.

2.  **<i class="fas fa-code"></i> Plot reconstruction error:** When constructing $\Phi$ you may use a single principal component or all of them. Plot the reconstruction error of $\Phi$ for all possible numbers of principle components. An example is shown in {numref}`fig:plot_var`.

3.  **<i class="fas fa-code"></i> Calculate variance:** Create functions that calculate the
    _proportional_ and _cumulative proportional_ variance.

4.  **<i class="fas fa-code"></i> Plot variance metrics:** Plot both the proportional and cumulative
    proportional variance in a single plot. An example is shown in
    {numref}`fig:plot_var`.


```{figure} ./img/plot_var.png
---
name: fig:plot_var
width: 400px
---
The expected result of the cumulative and individual variance
proportion.
```

In [20]:
# 1. Calculate reconstruction error
from math import sqrt

def error(X,k):
    
    # returns transformation of feature space to principal component given dataset after calculating  mean and pc
    x = pc_to_feature_space(X,k)
    rmse = np.sqrt(np.sum(np.power(X - x, 2))/120)
    return rmse
    
error(shapes,2)    
    

[[-0.1233574  -0.10821668 -0.10727042 ...  0.10262503  0.143708
   0.04485926]
 [-0.1233574  -0.10821668 -0.10727042 ...  0.10262503  0.143708
   0.04485926]
 [-0.1233574  -0.10821668 -0.10727042 ...  0.10262503  0.143708
   0.04485926]
 ...
 [-0.1233574  -0.10821668 -0.10727042 ...  0.10262503  0.143708
   0.04485926]
 [-0.1233574  -0.10821668 -0.10727042 ...  0.10262503  0.143708
   0.04485926]
 [-0.1233574  -0.10821668 -0.10727042 ...  0.10262503  0.143708
   0.04485926]]
[[-0.1233574  -0.10821668 -0.10727042 ...  0.10262503  0.143708
   0.04485926]
 [-0.1233574  -0.10821668 -0.10727042 ...  0.10262503  0.143708
   0.04485926]
 [-0.1233574  -0.10821668 -0.10727042 ...  0.10262503  0.143708
   0.04485926]
 ...
 [-0.1233574  -0.10821668 -0.10727042 ...  0.10262503  0.143708
   0.04485926]
 [-0.1233574  -0.10821668 -0.10727042 ...  0.10262503  0.143708
   0.04485926]
 [-0.1233574  -0.10821668 -0.10727042 ...  0.10262503  0.143708
   0.04485926]]


AttributeError: 'function' object has no attribute 'T'

In [ ]:
# Write your implementation here.

def variances(X,k):
    eigenvalues, eigenvectors, pc, B, mean_of_x, eig_pairs = pca(X,k) 

    eig_pairs = np.array(eig_pairs)

    sum_of_all_eig_values = 0

    for i in eig_pairs:
        sum_of_all_eig_values += i[0]

    cumulative_variance = []
    variance_proportion = []

    for i in range(120):
        tmp = 0
        variance_proportion.append(eig_pairs[i][0]/ sum_of_all_eig_values)

        for j in range(i):
            tmp += eig_pairs[j][0]

        cumulative_variance.append(tmp)

    plt.figure(figsize=(6, 4))
    plt.plot(variance_proportion, '-|', label="Variance proportion")
    plt.plot(cumulative_variance, '-|', label="Cumulative proportion")
    plt.legend()
    
print(variances(shapes,2))